In the previous Notebooks i've established that using a Linear regression to predict Rank is my best bet.

This method is called a pointwise ranking method. and the "learn to rank" Theory proposes more ranking methods that i'll try to explore in this norebook. 

    -Pairwise ranking 
    -Listwise ranking

Let's start with Pairwise ranking there are various methods all from Microsoft researcher Burges
RankNet, LambdaRank, LambdaRankMart

In [1]:
import pandas as pd
from scipy.stats import spearmanr
import numpy as np
from utils.Dataset_modification import *
from utils.compute_model_performance import *

X_train = pd.read_csv(datasets+'X_train.csv') #datasets is defined in utils
Y_train=pd.read_csv(datasets+'Y_train.csv')
Y_train['Rank']= Y_train['TARGET'].rank()
Y_train

,ID,TARGET,Rank
0,1054,0.028313,807.0
1,2049,-0.112516,532.0
2,1924,-0.180840,427.0
3,297,-0.260356,343.0
4,1101,-0.071733,593.0
...,...,...,...
1489,459,-0.172597,440.0
1490,1674,-0.063546,609.0
1491,748,0.151797,1016.0
1492,1454,-0.640917,163.0


In [21]:
df=create_pairwise_dataset(Y_train)
print(df)


            ID1     ID2  Comparison
0        1054.0  2049.0         1.0
1        1054.0  1924.0         1.0
2        1054.0   297.0         1.0
3        1054.0  1101.0         1.0
4        1054.0  1520.0         0.0
...         ...     ...         ...
1115266  1674.0  1454.0         1.0
1115267  1674.0   359.0         1.0
1115268   748.0  1454.0         1.0
1115269   748.0   359.0         1.0
1115270  1454.0   359.0         0.0

[1115271 rows x 3 columns]


In [22]:
import numpy.random as np_random
Comp_map = {1.0:np_random.uniform(0.66,1), 0.0: np_random.uniform(0,0.33), 0.5:np_random.uniform(0.33,0.66) } 
df['Comparison']=df['Comparison'].map(Comp_map)
df

,ID1,ID2,Comparison
0,1054.0,2049.0,0.970357
1,1054.0,1924.0,0.970357
2,1054.0,297.0,0.970357
3,1054.0,1101.0,0.970357
4,1054.0,1520.0,0.191529
...,...,...,...
1115266,1674.0,1454.0,0.970357
1115267,1674.0,359.0,0.970357
1115268,748.0,1454.0,0.970357
1115269,748.0,359.0,0.970357


In [23]:
remade=reconstruct_original_dataset(df)
remade

,ID,Rank_recreated
0,0.0,455
1,2.0,778
2,3.0,364
3,4.0,810
4,5.0,589
...,...,...
1489,2142.0,1357
1490,2143.0,1314
1491,2144.0,571
1492,2145.0,434


In [24]:
sorted=remade.sort_values(by=['ID'], key=lambda x: x.map({k: i for i, k in enumerate(Y_train['ID'].tolist())}))
sorted['ID'].tolist()==Y_train['ID'].tolist()

True

In [25]:
from scipy.stats import spearmanr
print("TARGET AND reconstructed Rank :" ,spearmanr(sorted["Rank_recreated"], Y_train["TARGET"]).correlation)


TARGET AND reconstructed Rank : 0.9999998515590814


Predicting Pairwise ranking accurately -> predicting Ranking accurately -> high spearman score

meaning our method of creating pairwise ranking and prediction a pairwise ranking ID by ID might help determine a more accurate ranking. Now let's Get a  X_train_pairwise that matches our Y_train_Pairwise

In [6]:
X_train_pairwise=get_training_pairwise(X_train)

Country remapped


In [13]:
print(sorted(Y_train['TARGET'].tolist(),reverse=True)[:10])

[7.786577856046542, 7.138604025388376, 6.836019876868637, 6.681753850368199, 6.151133366269985, 6.094297093901249, 6.009904429692065, 5.57011518732641, 5.0164912760257, 5.015579833097478]


In [1]:
import pandas as pd
from scipy.stats import spearmanr
import numpy as np
from utils.Dataset_modification import *
from utils.compute_model_performance import *

X_train = pd.read_csv(datasets+'X_train.csv') #datasets is defined in utils
Y_train=pd.read_csv(datasets+'Y_train.csv')

from sklearn.linear_model import LinearRegression
from utils.compute_model_performance import *

model=Model(LinearRegression,LinearRegression)
model.get_data_and_order(X_train,Y_train) 
# model.Compute(iterations=100,custom_group=2,group=True)
model.Compute(iterations=1,columns='Rank_recreated',split=False)



X_train and Y_train same ID ? True
X_test and Y_test same ID ? True
Country remapped
Country remapped
pairwise creation done
list length is 69751
error_val (0.03487844588601563, [2.12197347021395, 2.1171241082318657, 2.051565838784735, 2.047340904632818, 1.9967309744246213, 1.9825621361476666, 1.9481150934506246, 1.940040703897176, 1.935191341915091, 1.935083837448751, 1.9319930185943597, 1.9236743218718537, 1.921064752781457, 1.9108217275462605, 1.9021964383517564, 1.900966597654162, 1.8967282698922778, 1.8966815572643538, 1.8953863368053414, 1.8876776549537981, 1.8853187543153809, 1.8762663259383512, 1.8738773253348393, 1.86963307246796, 1.8654081383160435, 1.8637120251152044, 1.8445358292522522, 1.8421158752682398, 1.8363204403376936, 1.8312784621554572, 1.8312775049114867, 1.8311490679557378, 1.8296232038282936, 1.8288926365552842, 1.8264546784456053, 1.8256197962683585, 1.823958770675031, 1.8227686891078392, 1.8222677529392288, 1.8213424809174414, 1.8212799115985836, 1.81849542066

In [2]:


from sklearn.linear_model import LogisticRegression
from utils.compute_model_performance import *

model=Model(LogisticRegression,LogisticRegression)
model.get_data_and_order(X_train,Y_train) 
# model.Compute(iterations=100,custom_group=2,group=True)
model.Compute(iterations=2,columns='Rank_recreated',split=False)



X_train and Y_train same ID ? True
X_test and Y_test same ID ? True
Country remapped
Country remapped
pairwise creation done


/home/ymachta/miniconda3/envs/qube/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


list length is 69751
error_val (-0.06846496824418288, [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])
accuracy_val [0.5761781193101174, 0.6379914704679558, 0.5166122311071066]
list length is 626640
error_train (-0.026073981871569004, [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1

/home/ymachta/miniconda3/envs/qube/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


list length is 69751
error_val (-0.06108872991068228, [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])
accuracy_val [0.5640635976545139, 0.6177708905911935, 0.5123913471546316]
list length is 626640
error_train (-0.026129835312140942, [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1

In [5]:
from mord import OrdinalRidge
from utils.compute_model_performance import *
model=Model(OrdinalRidge,OrdinalRidge)
model.get_data_and_order(X_train,Y_train) 
model.Compute(iterations=2,columns='Rank_recreated',split=False)

X_train and Y_train same ID ? True
X_test and Y_test same ID ? True
Country remapped
Country remapped
pairwise creation done


/home/ymachta/miniconda3/envs/qube/lib/python3.11/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but OrdinalRidge was fitted without feature names
  warnings.warn(
/home/ymachta/miniconda3/envs/qube/lib/python3.11/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but OrdinalRidge was fitted without feature names
  warnings.warn(


list length is 69751
error_val (-0.049913262892288285, [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])
accuracy_val 0.5784003096729796
list length is 626640
error_train (0.003501212817566705, [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0

/home/ymachta/miniconda3/envs/qube/lib/python3.11/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but OrdinalRidge was fitted without feature names
  warnings.warn(
/home/ymachta/miniconda3/envs/qube/lib/python3.11/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but OrdinalRidge was fitted without feature names
  warnings.warn(


list length is 69751
error_val (-0.02602830066952445, [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])
accuracy_val 0.572808991985778
list length is 626640
error_train (-0.017856344950848972, [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,

In [2]:
from sklearn.ensemble import RandomForestRegressor
model=Model(RandomForestRegressor,RandomForestRegressor,depth=10,n_estimators=2)
model.get_data_and_order(X_train,Y_train) 
model.Compute(iterations=1,columns='Rank_recreated',split=False)

X_train and Y_train same ID ? True
X_test and Y_test same ID ? True
Country remapped
Country remapped
pairwise creation done
list length is 69751
error_val (0.020804848536370525, [2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.995382585751979, 1.9641786109305692, 1.9641786109305692, 1.9641786109305692, 1.9641786109305692, 1.9641786109305692, 1.954921803127875, 1.954921803127875, 1.954921803127875, 1.954921803127875, 1.954921803127875, 1.9405391349570946, 1.9405391349570946, 1.9303952917188005, 1.9303952917188005, 1.9303952917188005, 1.9303952917188005, 1.9303952917188005, 1.9303952917188005, 1.930131004366812, 1.9270386266094421, 1.9270386266094421, 1.9270386266094421, 1.9161490683229814, 1.9063694963399977, 1.9063694963399977, 1.9063694963399977, 1.9063694963399977, 1.9063694963399977, 1.9063694963399977, 1.905140904868599, 1.905140904868599, 1.905140904868599, 1.905140904868599, 1.905140904868599, 1.905140904868599, 1.905140904868599, 1.905140904868599, 1.905140904868599, 1.905140904

In [2]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from utils.compute_model_performance import *

# model=Model(RandomForestRegressor,RandomForestRegressor,depth=3,n_estimators=10)
# model.get_data_and_order(X_train,Y_train) 
# model.Compute(iterations=1,columns='Rank_recreated',split=False)

model=Model(RandomForestClassifier,RandomForestClassifier,depth=10,n_estimators=100)
model.get_data_and_order(X_train,Y_train) 
model.Compute(iterations=1,columns='Rank_recreated',split=False)

X_train and Y_train same ID ? True
X_test and Y_test same ID ? True
Country remapped
Country remapped
pairwise creation done
error_val 0.12432796662413442
accuracy_val 0.5497412223480667
error_train 0.03807050300012767
accuracy_train 0.8162980339588919
reconstructing rank ...
Y_pred and Y_test same ID ? True
Same length ? True
         ID  Rank_recreated    ID    TARGET    Rank
0    1405.0             302  1405  0.246768  1107.0
1    1080.0             373  1080  0.001998   760.0
2    1846.0             373  1846 -1.155310    74.0
3    1199.0              59  1199 -0.637658   165.0
4    1639.0             321  1639 -0.121041   515.0
..      ...             ...   ...       ...     ...
369  1328.0              80  1328 -0.205207   389.0
370   711.0              80   711  1.017795  1365.0
371   627.0             373   627 -2.567250    15.0
372   914.0             373   914  1.732405  1426.0
373   456.0             373   456  1.608778  1419.0

[374 rows x 5 columns]
validation 0.5586297375

In [2]:
import xgboost as xgb

model=Model(xgb.XGBClassifier,xgb.XGBClassifier,depth=10,n_estimators=50)
model.get_data_and_order(X_train,Y_train) 
model.Compute(iterations=1,columns='Rank_recreated',split=False)

X_train and Y_train same ID ? True
X_test and Y_test same ID ? True
Country remapped
Country remapped
pairwise creation done


TypeError: 'NoneType' object is not subscriptable